1.Lets install and invoke packages.

In [1]:
!pip install flair
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertForTokenClassification
import requests,os
from flair.datasets import ColumnCorpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print(os.getenv("PYTORCH_CUDA_ALLOC_CONF"))

/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


expandable_segments:True


2-Loading tokenizers, models, etc

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=9)
train_url = 'https://raw.githubusercontent.com/lang-uk/flair-ner/main/fixed-split/train.iob'
test_url = 'https://raw.githubusercontent.com/lang-uk/flair-ner/main/fixed-split/test.iob'

def download_data(url, file_name):
    if not os.path.exists(file_name):
        response = requests.get(url)
        with open(file_name, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print(f'Data downloaded: {file_name}')
    else:
        print('Data found:', file_name)
data_folder = './data/'
os.makedirs(data_folder, exist_ok=True)
download_data(train_url, os.path.join(data_folder, 'train.iob'))
download_data(test_url, os.path.join(data_folder, 'test.iob'))

distilbert_embeddings = TransformerWordEmbeddings('distilbert-base-multilingual-cased', fine_tune=True)


/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data found: ./data/train.iob
Data found: ./data/test.iob


Lets make corpus

In [3]:
columns = {0: 'text', 1: 'ner'}
corpus = ColumnCorpus(data_folder, columns,
                      train_file='train.iob',
                      test_file='test.iob')
print(corpus)
for i in range(15):
    print(corpus.train[i])
tag_types = corpus.make_label_dictionary(label_type='ner')
print(tag_types)




2024-09-07 18:49:07,654 Reading data from data
2024-09-07 18:49:07,655 Train: data/train.iob
2024-09-07 18:49:07,656 Dev: None
2024-09-07 18:49:07,656 Test: data/test.iob
2024-09-07 18:49:10,234 No dev split found. Using 10% (i.e. 876 samples) of the train split as dev data
Corpus: 7886 train + 876 dev + 4045 test sentences
Sentence[22]: "З одного боку , саме через це більшість проектів КСВ здійснюються епізодично та деколи виглядають , радше , як просто благодійність ."
Sentence[8]: "Винятком будуть хіба що представництва іноземних корпорацій ."
Sentence[20]: "З іншого боку , для українських компаній відкривається потужне « вікно можливостей » , щоб втілювати власні унікальні стратегії ."
Sentence[3]: "Філантропія як пріоритет"
Sentence[17]: "Найпоширенішим підходом українського бізнесу у КСВ є філантропія — компанії виділяють кошти на суспільно важливі проекти ."
Sentence[15]: "Це може бути як власна ініціатива , так і відповідь на запит від громади ."
Sentence[17]: "Це абсолютно нор

8it [00:00, 7612.17it/s]
7886it [00:00, 32118.20it/s]

2024-09-07 18:49:10,487 Dictionary created for label 'ner' with 4 values: PERS (seen 2473 times), LOC (seen 1101 times), ORG (seen 470 times), MISC (seen 381 times)
Dictionary with 4 tags: PERS, LOC, ORG, MISC


Lets make tagger and trainer

In [4]:
tagger = SequenceTagger(hidden_size=128,
                        embeddings=distilbert_embeddings,
                        tag_dictionary=tag_types,
                        tag_type='ner',
                        use_crf=False)
trainer = ModelTrainer(tagger, corpus)


2024-09-07 18:49:10,496 SequenceTagger predicts: Dictionary with 17 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC


Model name and and model

In [5]:
finetuned_model_name='dist-finetuned.04'


In [6]:
trainer.train(finetuned_model_name,
              learning_rate=0.01,
              mini_batch_size=1,
              max_epochs=10,
              embeddings_storage_mode='cpu',
              #main_evaluation_metric='f1',
              monitor_test=True,
              #use_amp=True,
              train_with_dev=True) 

2024-09-07 18:49:14,819 ----------------------------------------------------------------------------------------------------
2024-09-07 18:49:14,821 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(119548, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_featu

/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-09-07 18:50:11,959 epoch 1 - iter 876/8762 - loss 0.34582672 - time (sec): 57.12 - samples/sec: 280.40 - lr: 0.010000 - momentum: 0.000000
2024-09-07 18:51:01,790 epoch 1 - iter 1752/8762 - loss 0.27080861 - time (sec): 106.95 - samples/sec: 301.64 - lr: 0.010000 - momentum: 0.000000
2024-09-07 18:51:51,703 epoch 1 - iter 2628/8762 - loss 0.23174478 - time (sec): 156.86 - samples/sec: 308.49 - lr: 0.010000 - momentum: 0.000000
2024-09-07 18:52:42,026 epoch 1 - iter 3504/8762 - loss 0.20704101 - time (sec): 207.18 - samples/sec: 306.43 - lr: 0.010000 - momentum: 0.000000
2024-09-07 18:53:32,059 epoch 1 - iter 4380/8762 - loss 0.19170939 - time (sec): 257.22 - samples/sec: 308.24 - lr: 0.010000 - momentum: 0.000000
2024-09-07 18:54:21,711 epoch 1 - iter 5256/8762 - loss 0.18353368 - time (sec): 306.87 - samples/sec: 310.00 - lr: 0.010000 - momentum: 0.000000
2024-09-07 18:55:12,066 epoch 1 - iter 6132/8762 - loss 0.17462399 - time (sec): 357.22 - samples/sec: 311.93 - lr: 0.010000 -

100%|██████████| 64/64 [01:03<00:00,  1.00it/s]


2024-09-07 18:58:49,538 TEST : loss 0.14886006712913513 - f1-score (micro avg)  0.6764
2024-09-07 18:58:49,754  - 0 epochs without improvement
2024-09-07 18:58:49,790 ----------------------------------------------------------------------------------------------------
2024-09-07 18:59:40,547 epoch 2 - iter 876/8762 - loss 0.10313069 - time (sec): 50.76 - samples/sec: 315.49 - lr: 0.010000 - momentum: 0.000000
2024-09-07 19:00:31,022 epoch 2 - iter 1752/8762 - loss 0.11191604 - time (sec): 101.23 - samples/sec: 308.35 - lr: 0.010000 - momentum: 0.000000
2024-09-07 19:01:22,178 epoch 2 - iter 2628/8762 - loss 0.10588968 - time (sec): 152.39 - samples/sec: 308.26 - lr: 0.010000 - momentum: 0.000000
2024-09-07 19:02:13,430 epoch 2 - iter 3504/8762 - loss 0.10215125 - time (sec): 203.64 - samples/sec: 307.17 - lr: 0.010000 - momentum: 0.000000
2024-09-07 19:03:04,859 epoch 2 - iter 4380/8762 - loss 0.10771823 - time (sec): 255.07 - samples/sec: 311.04 - lr: 0.010000 - momentum: 0.000000
2024

...

2024-09-07 20:16:37,084 TEST : loss 0.20052605867385864 - f1-score (micro avg)  0.7083


2024-09-07 20:16:37,300  - 0 epochs without improvement
2024-09-07 20:16:44,948 ----------------------------------------------------------------------------------------------------
2024-09-07 20:16:44,971 Testing using last state of model ...


100%|██████████| 64/64 [00:06<00:00,  9.62it/s]


2024-09-07 20:16:51,815 
Results:
- F-score (micro) 0.7083
- F-score (macro) 0.4936
- Accuracy 0.6273

By class:
              precision    recall  f1-score   support

        PERS     0.9090    0.8278    0.8665      1678
         ORG     0.2791    0.6705    0.3941       261
         LOC     0.6410    0.7257    0.6807       401
        MISC     0.0794    0.0208    0.0330       240

   micro avg     0.6961    0.7209    0.7083      2580
   macro avg     0.4771    0.5612    0.4936      2580
weighted avg     0.7265    0.7209    0.7123      2580

2024-09-07 20:16:51,816 ----------------------------------------------------------------------------------------------------


{'test_score': 0.7083015993907081}

Lets evaluate it

In [7]:
result = tagger.evaluate(corpus.test, gold_label_type='ner')  # Evaluate on the test set
print(result.detailed_results)

100%|██████████| 127/127 [00:50<00:00,  2.52it/s]


Results:
- F-score (micro) 0.8018
- F-score (macro) 0.6901
- Accuracy 0.7308

By class:
              precision    recall  f1-score   support

        PERS     0.9152    0.8743    0.8942      1678
         LOC     0.7127    0.7855    0.7473       401
        MISC     0.4641    0.5917    0.5201       240
         ORG     0.6120    0.5862    0.5988       261

   micro avg     0.7985    0.8050    0.8018      2580
   macro avg     0.6760    0.7094    0.6901      2580
weighted avg     0.8111    0.8050    0.8067      2580



Lets test with own data

In [15]:
finetuned_model = SequenceTagger.load('/home/sginne/src/master_thesis/colab/dist-finetuned.04/final-model.pt')
def print_tagged_sentences(sentences, model=None):
    # Load model only if not already loaded
    
    for text in sentences:
        sentence = Sentence(text)  # Create Sentence object
        model.predict(sentence)    # Predict tags
        #print (sentence.to_tagged_string())
        tagged_sentence = []
        spans=sentence.get_spans('ner')
    
        #for span in spans:
        #    print ("Span:",span.get_label('ner').value)
        #    print ("Span:",span.get_label('ner').score)
        i=0
        for token in sentence:
            if i!=len(spans):
                if (token==spans[i][0]):
                    current_tag_token=spans[i]
                    current_tag_value=current_tag_token.get_label('ner').value
                    current_tag_score=current_tag_token.get_label('ner').score
                    tagged_sentence.append(f'[{token.text}|({current_tag_value}={current_tag_score*100:.1f}%)]')
                    i=i+1
                else:
                    tagged_sentence.append(token.text)
            else:
                tagged_sentence.append(token.text)

        print ( ' '.join(tagged_sentence))

sentences = [
    'Привіт , мене звати Тімо і я живу в Турку , навчаюсь в Универсітеті Турку',
    'Тімо Юнолайнен живе в місті Турку й навчається в Університеті міста Турку.',
    'Місто Турку розташоване на південно-західному узбережжі Фінляндії.',
    'В Турку є поліція, й вона вельми корумпована',
    'У Турку проходить щорічний фестиваль музики та мистецтв.',
    'Тімо Юнолайнен, професор в Університеті міста Турку, співпрацює з організацією "Турку Інститут" для розробки нових проектів у Турку.'
]

print_tagged_sentences(sentences, finetuned_model)

2024-09-07 20:25:58,049 SequenceTagger predicts: Dictionary with 17 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC
Привіт , мене звати [Тімо|(PERS=100.0%)] і я живу в [Турку|(LOC=100.0%)] , навчаюсь в [Универсітеті|(ORG=90.3%)] Турку
Тімо [Юнолайнен|(PERS=100.0%)] живе в місті [Турку|(LOC=100.0%)] й навчається в [Університеті|(ORG=99.7%)] міста Турку .
Місто [Турку|(LOC=100.0%)] розташоване на південно-західному узбережжі [Фінляндії|(LOC=100.0%)] .
В [Турку|(LOC=99.8%)] є поліція , й вона вельми корумпована
У [Турку|(LOC=100.0%)] проходить щорічний фестиваль музики та мистецтв .
Тімо [Юнолайнен|(PERS=93.4%)] , професор в [Університеті|(ORG=99.6%)] міста Турку , співпрацює з організацією " [Турку|(ORG=99.5%)] Інститут " для розробки нових проектів у [Турку|(LOC=100.0%)] .
